In [13]:
import customtkinter as ctk
import tkinter.messagebox as messagebox
import json, os
from datetime import datetime

FILENAME = "tasks.json"

class ToDoApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("📝 To-Do List")
        self.geometry("500x600")
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("dark-blue")

        self.tasks = []

        # Header
        header = ctk.CTkLabel(self, text="To-Do List", font=("Segoe UI", 24, "bold"))
        header.pack(pady=(20, 10))

        # Task Frame
        self.task_frame = ctk.CTkFrame(self, corner_radius=10)
        self.task_frame.pack(fill="both", expand=True, padx=20, pady=10)

        self.task_listbox = ctk.CTkTextbox(self.task_frame, height=360, font=("Segoe UI", 14), wrap="word", state="disabled")
        self.task_listbox.pack(fill="both", expand=True, padx=10, pady=10)

        # Button Panel
        button_panel = ctk.CTkFrame(self)
        button_panel.pack(pady=10)

        ctk.CTkButton(button_panel, text="Add", width=90, command=self.add_task, fg_color="#00FFB7").grid(row=0, column=0, padx=6)
        ctk.CTkButton(button_panel, text="Edit", width=90, command=self.edit_task, fg_color="#FFA500").grid(row=0, column=1, padx=6)
        ctk.CTkButton(button_panel, text="Delete", width=90, command=self.delete_task, fg_color="#FF4C4C").grid(row=0, column=2, padx=6)
        ctk.CTkButton(button_panel, text="Done", width=90, command=self.mark_done, fg_color="#90EE90").grid(row=0, column=3, padx=6)

        self.load_tasks()

    def refresh_display(self):
        self.task_listbox.configure(state="normal")
        self.task_listbox.delete("1.0", "end")

        for i, task in enumerate(self.tasks):
            status = "✔️" if task["done"] else "❗"
            emoji = {"High": "🔥", "Medium": "🔷", "Low": "🟢"}.get(task["priority"], "⬜")
            due = f"(Due: {task['due_date']})"
            line = f"{i+1}. {status} {emoji} {task['task']} {due}\n"
            self.task_listbox.insert("end", line)

        self.task_listbox.configure(state="disabled")

    def add_task(self):
        task = ctk.CTkInputDialog(title="New Task", text="Enter task:").get_input()
        if not task:
            return
        priority = ctk.CTkInputDialog(title="Priority", text="Priority (High/Medium/Low):").get_input()
        due = ctk.CTkInputDialog(title="Due Date", text="Due Date (YYYY-MM-DD):").get_input()

        self.tasks.append({
            "task": task,
            "done": False,
            "priority": priority or "Medium",
            "due_date": due or datetime.now().strftime('%Y-%m-%d')
        })
        self.refresh_display()
        self.save_tasks()

    def edit_task(self):
        index = self.select_index()
        if index is None: return

        task_data = self.tasks[index]
        new_task = ctk.CTkInputDialog(title="Edit Task", text="Update task:").get_input()
        if new_task:
            priority = ctk.CTkInputDialog(title="Edit Priority", text="Priority (High/Medium/Low):").get_input()
            due = ctk.CTkInputDialog(title="Edit Due Date", text="Due Date (YYYY-MM-DD):").get_input()

            self.tasks[index] = {
                "task": new_task,
                "done": task_data["done"],
                "priority": priority or "Medium",
                "due_date": due or task_data["due_date"]
            }
            self.refresh_display()
            self.save_tasks()

    def delete_task(self):
        index = self.select_index()
        if index is not None:
            del self.tasks[index]
            self.refresh_display()
            self.save_tasks()

    def mark_done(self):
        index = self.select_index()
        if index is not None:
            self.tasks[index]["done"] = not self.tasks[index]["done"]
            self.refresh_display()
            self.save_tasks()

    def select_index(self):
        try:
            index = int(simpledialog.askstring("Task Index", "Enter task number:")) - 1
            if 0 <= index < len(self.tasks):
                return index
            else:
                messagebox.showerror("Invalid", "Task number out of range.")
        except:
            messagebox.showerror("Error", "Please enter a valid number.")
        return None

    def save_tasks(self):
        with open(FILENAME, "w") as f:
            json.dump(self.tasks, f, indent=2)

    def load_tasks(self):
        if os.path.exists(FILENAME):
            with open(FILENAME, "r") as f:
                self.tasks = json.load(f)
                self.refresh_display()

# Run the app
if __name__ == "__main__":
    app = ToDoApp()
    app.mainloop()

   



